In [12]:
import glob
import os

In [13]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.schema import Document
import os


loader = PyPDFDirectoryLoader("DIC")
documents = loader.load()

print(len(documents))

175


In [14]:
from dotenv import load_dotenv; 
load_dotenv(); 
import os;
pinecone_api_key = os.getenv('PINECONE_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')


In [15]:
len([f for f in os.listdir("./DIC/")])

64

In [16]:
documents[4]

Document(metadata={'source': 'DIC/FR0007040373_DIC_FR_20230630.pdf', 'page': 0}, page_content="DOCUMENT D’INFORMATIONS CLÉS \nLe présent document contient des informations essentielles sur le produit d’investissement. Il ne s’agit pas d’un document à caractère commercial. \nCes informations vous sont fournies conformément à une obligation légale, afin de vous aider à comprendre en quoi consiste ce p roduit et quels \nrisques, coûts, gains et pertes potentiels y sont associés, et de vous aider à le comparer à d’autres produits.\nEn quoi consiste ce produit ? \nType : Fonds d'Investissement à Vocation Générale (FIVG) \n \nDurée\n : Le Fonds est créé pour une durée de 99 ans. \n \nObjectifs\n :   \nL’objectif du FCP est, au travers d’une gestion discrétionnaire, la recherche d’une \nperformance sur une durée de placement recommandée de 5 ans, en s’exposant aux \nmarchés financiers français ou étrangers actions, obligations et autres titres de créances \npar le biais d’investissements en t

In [17]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer


# Charger le modèle d'encodage de texte BAAI/bge-small-en-v1.5 de HuggingFace
#embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", encode_kwargs={"normalize_embeddings" : True})
embedding = HuggingFaceEmbeddings(model_name="HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1", encode_kwargs={"normalize_embeddings" : True})


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_experimental.text_splitter import SemanticChunker


text_splitter = SemanticChunker(embedding)

# Division du document en morceaux (chunks)
chunks = text_splitter.split_documents(documents=documents)

# Affichage du nombre de morceaux créés à partir du document PDF
print(f"{len(chunks)} chunks ont été créés par le splitter à partir du document PDF.")

534 chunks ont été créés par le splitter à partir du document PDF.


In [19]:
chunks[500]

Document(metadata={'source': 'DIC/fidelity.pdf', 'page': 1}, page_content="Des informations sur l es références \nantérieures figurent dans le rapport et les comptes  annuels. 12.2 \n22.8 \n33.8 \n21.5 19.8 19.4 \n0.5 \n46.5 \n32.3 31.5 \n13.6 \n21.1 \n31.2 \n15.0 15.6 \n24.5 \n-1.1 \n49.6 \n33.6 37.0 \n-10% \n0% \n10% \n20% \n30% \n40% \n50% \n60% \n2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 \nLes frais d'entrée et de sortie communiqués sont de s chiffres maximum. Dans \ncertains cas, vous pouvez être amené à payer moins.")

In [20]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.exceptions import NotFoundException

# TODO: Inscrire la clé API Pinecone
pinecone = Pinecone(api_key=pinecone_api_key)

pinecone.delete_index("rag")
pinecone.create_index("rag", dimension=896, spec=ServerlessSpec(cloud="aws", region="us-east-1"))


In [21]:
from langchain_pinecone.vectorstores import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec


pinecone = Pinecone(api_key=pinecone_api_key)


# Initialiser le VectorStore de LlamaIndex avec l'index de Pinecone
pinecone_index = pinecone.Index("rag")
vector_store = PineconeVectorStore(
    index=pinecone_index,
    embedding=embedding
)

In [22]:
#add_result = vector_store.add_documents(chunks)
# 1. Filter the list to remove invalid documents
print(f"Original number of chunks: {len(chunks)}")
cleaned_chunks = [doc for doc in chunks if doc.page_content.strip()]
print(f"Number of chunks after cleaning: {len(cleaned_chunks)}")

# 2. Add only the valid, cleaned documents to the vector store
if cleaned_chunks:
    add_result = vector_store.add_documents(cleaned_chunks)
else:
    print("No valid chunks to add after cleaning.")
print(f"{len(add_result)} vecteurs ont été ajoutés dans Pinecone.")

Original number of chunks: 534
Number of chunks after cleaning: 529
529 vecteurs ont été ajoutés dans Pinecone.


In [23]:
len(add_result)

529

In [24]:
!huggingface-cli login --token={huggingface_api_key}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [25]:
import transformers
import torch

from transformers import BitsAndBytesConfig

model_id = "microsoft/Phi-3-mini-4k-instruct"
#model_id = "curiousily/Llama-3-8B-Instruct-Finance-RAG"


# Chargement de la configuration du modèle

# Initialiser le tokeniseur
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map='auto'
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
from langchain_huggingface.llms import HuggingFacePipeline

from transformers import pipeline

llm=HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9,
        truncation=True
    )
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",   # ou "gemini-1.5-flash" si tu veux aller plus vite
    temperature=0.0
)

kg_transformer = LLMGraphTransformer(llm=llm)
graph_docs = kg_transformer.convert_to_graph_documents(cleaned_chunks)

In [ ]:
import networkx as nx
G = nx.MultiDiGraph()

for gdoc in graph_docs:
    for rel in gdoc.relationships:
        src = rel.source
        dst = rel.target
        G.add_edge(src.id, dst.id, label=rel.type)

In [ ]:
def query_graph(G, question, k_hops=2, max_edges=50):
    # 1. repérage des entités probables dans la question
    terms = [t.lower() for t in question.split() if len(t) > 3]
    matched_nodes = [n for n in G.nodes if any(t in str(n).lower() for t in terms)]

    sub_edges = set()
    for node in matched_nodes:
        # explore les voisins proches
        for src, dst, data in nx.bfs_edges(G, node, depth_limit=k_hops):
            sub_edges.add((src, dst, data.get("label", "RELATED")))

    # tronquer pour éviter explosion
    sub_edges = list(sub_edges)[:max_edges]

    # transformer en texte “triples” utilisables par le LLM
    triples_text = "\n".join([f"({s}) -[{r}]-> ({d})" for s, d, r in sub_edges])
    return triples_text


In [ ]:
def query_vectorstore(vectorstore, question, k=5):
    docs = vectorstore.similarity_search(question, k=k)
    return "\n\n".join(d.page_content for d in docs)


In [ ]:
def build_context(G, vectorstore, question):
    graph_context = query_graph(G, question)
    vector_context = query_vectorstore(vectorstore, question)
    return f"""
=== CONTEXTE GRAPHE ===
{graph_context}

=== CONTEXTE TEXTE ===
{vector_context}
"""


In [ ]:
import torch

def answer_with_hybrid_rag(G, vectorstore, question, model, tokenizer, max_new_tokens=512):
    context = build_context(G, vectorstore, question)

    prompt = f"""
Tu es un assistant RAG hybride.
Contexte provenant d'un graphe de connaissances et d'une recherche vectorielle :
{context}

Question : {question}
Réponds en une phrase de manière factuelle et concise en t’appuyant uniquement sur ces éléments.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.2,
            do_sample=False
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer



In [7]:
# Exemple
response = answer_with_hybrid_rag(G, vector_store, "A quoi vise la prise en compte des critères ESG dans la sélection de titres du FCP?", model, tokenizer)
print(response)


NameError: name 'G' is not defined

In [21]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""Provide a short and precise answer to the following question, based solely on the information from the documents below:

---------------------
{context}
---------------------

Use the information in these documents to answer the question in a factual and concise manner. If the answer to the question is not contained within these documents, respond simply with "Unknown":

Question: {question}""")

def build_context(docs):
    context = ""
    for doc in docs:
        context += "titre du document :" + doc.metadata["source"]
        context += "\n"
        context += doc.page_content
        context += "\n\n"
    print(context)
        
def rag_pipeline(query):
    # Tout d'abord, on recherche les documents
    retrieved_docs = vector_store.similarity_search(query)
    # Ensuite, on injecte les documents dans le prompt
    prompt = prompt_template.invoke({
        "question": query,
        "context": build_context(retrieved_docs)
    })
    # Enfin, on envoit l'intégralité du prompt au LLM
    print(build_context(retrieved_docs))
    return llm.invoke(prompt).strip()

In [23]:
query = """
Comment puis-je formuler une réclamation?
"""

# Effectuer une requête
response = rag_pipeline(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


titre du document :DIC/KIDPRIIPs_448092_79629_FRA_FRA_20240101.pdf
» pour plus de détails sur les frais de sortie. Comment puis-je formuler une réclamation ?Si vous avez des réclamations, vous pouvez :Appeler notre hotline dédiée aux réclamations au +352 2686 8001Envoyer un courrier à Amundi Luxembourg S.A., Service clients, 5 allée Scheﬀer, 2520 Luxembourg, LuxembourgEnvoyer un e-mail à info@amundi.com Dans le cas d’une réclamation, vous devez indiquer clairement vos coordonnées (nom, adresse, numéro de téléphone ou adresse e-mail) et fournir unebrève explication de votre réclamation. Vous trouverez davantage d’informations sur notre site Internet www.amundi.lu.

titre du document :DIC/KIDPRIIPs_447577_79682_FRA_FRA_20240101.pdf
» pour plus de détails sur les frais de sortie. Comment puis-je formuler une réclamation ?Si vous avez des réclamations, vous pouvez :Appeler notre hotline dédiée aux réclamations au +352 2686 8001Envoyer un courrier à Amundi Luxembourg S.A., Service clients, 

# TO DO

### Eval
### Modifier le modele d'Embedding
### reprompt de la requete utilisateur
### Algo de retrieval , recherche hybride ou mot clefs (VM 25)

### Graph Rag
https://microsoft.github.io/graphrag/
### fabrication d'arbre de connaissance en cluster
https://github.com/parthsarthi03/raptor